# 🧊 Trabajo Práctico Integrador – Parte 2  
## Transformación y creación de la capa Silver (Weather + Air Quality)

En esta segunda parte procesamos los datos horarios provenientes de la **capa Bronze** para generar datasets limpios y estandarizados en la **capa Silver**, usando **pandas**, **Delta Lake** y **MinIO (S3)**.  
Se incluyen tanto los datos de **Weather** como los de **Air Quality**.

---

## 🎯 Objetivos
- Leer datos de Bronze (Weather + Air Quality).  
- Limpiar, transformar y enriquecer ambos datasets.  
- Crear un Silver horario y un Silver agregado diario.  
- Guardar ambos datasets nuevamente en Delta Lake.

---

## 🔧 Proceso de Transformación

### 🧾 Lectura desde Bronze
Cargamos los datasets:
- `weather_hourly`  
- `air_quality_hourly`  

> Si falla la lectura, se retorna un DataFrame vacío para continuar el flujo sin errores.

### 🧹 Limpieza de datos
- Columnas numéricas → rellenar nulos con la mediana  
- Columnas categóricas → rellenar nulos con `'unknown'`  
- No se eliminan filas completas (no hay muchos nulos)

### 🧽 Eliminación de duplicados
Se remueven registros repetidos.  
- Si existen `time` + `station_id` (Weather) o `time` + `location` (Air Quality), se usan como clave para identificar duplicados.

### 🕒 Conversión de fechas y tipos
- Se convierte la columna `time` a datetime  
- Se crean columnas auxiliares:  
  - `extract_date` → fecha (`YYYY-MM-DD`)  
  - `extract_hour` → hora (`HH`)  
- Se optimizan columnas categóricas (`id`, `station_id`, `location`, `name`)

### 🏷️ Renombrado
Estandarización (solo Weather por ahora):  
- `temperature` → `temp_c`  
- `humidity` → `rel_humidity_pct`

### ➕ Columnas derivadas
Se agregan variables útiles:  
`temp_above_30`, `temp_f`, `dew_point_c_est`, `rolling_mean_temp_3h`, `heat_index_c`, `wind_kmh`, `temp_category`, `period_day`, `weekday`.

### 📈 Agregación diaria
- Weather: promedios, medianas, máximos, mínimos, viento promedio, horas con >30°C y horas con lluvia  
- Air Quality: promedios diarios de PM10, PM2.5, CO, O3, NO2 y SO2

---

## 🪙 Guardado en Silver
Se generan cuatro datasets:
- **Silver horario Weather**  
- **Silver horario Air Quality**  
- **Silver diario Weather**  
- **Silver diario Air Quality**  

> Modo de escritura: **overwrite**, para mantener siempre la versión más reciente y coherente.

---

## 🔒 Seguridad y manejo de credenciales
- Todas las credenciales de MinIO/S3 se gestionan mediante variables de entorno o un archivo `.env`.  
- No se exponen claves ni contraseñas directamente en el código.  
- Esto asegura que el código pueda compartirse o versionarse sin comprometer datos sensibles.


### 🛠️ Instalación de librerías

En esta celda instalamos todas las dependencias necesarias para el TP:

- **requests** → para consumir la API Open-Meteo  
- **pandas** → para manipular DataFrames  
- **pyarrow** → soporte columnar para Delta Lake  
- **deltalake** → para escribir y leer tablas Delta directamente en MinIO  
- **python-dotenv** → para cargar variables de entorno desde un archivo `.env`  

> Esta instalación debe ejecutarse **una sola vez** al iniciar el entorno.


In [1]:
!pip install requests pandas pyarrow deltalake python-dotenv --quiet
print("✔ Librerías instaladas correctamente")


✔ Librerías instaladas correctamente


### 🔒 Carga de variables de entorno

En esta celda:

- Se carga un archivo `.env` para mantener **seguras las credenciales** y rutas sensibles.
- Se asignan variables a Python desde el archivo `.env`.
- Esto permite que el código del notebook no contenga claves hardcodeadas y sea más seguro y portable.


In [2]:
# ===============================================================
# 🔑 Carga de variables de entorno
# ===============================================================

from dotenv import load_dotenv
import os

# Cargar archivo .env
load_dotenv()

# Asignar variables de entorno
AWS_ENDPOINT_URL = os.environ["AWS_ENDPOINT_URL"]
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
BRONZE_FULL = os.environ["BRONZE_FULL"]
BRONZE_INC = os.environ["BRONZE_INC"]
SILVER_WEATHER = os.environ["SILVER_WEATHER"]
SILVER_DAILY_AVG = os.environ["SILVER_DAILY_AVG"]

print("Variables de entorno cargadas correctamente")


Variables de entorno cargadas correctamente


### 🔧 Configuración de MinIO y rutas

En esta celda:

- Se conecta a MinIO (S3) usando las credenciales cargadas desde `.env`.
- Se definen las rutas para la capa Bronze y Silver.
- De esta forma, el código queda seguro y portable entre distintos entornos.


In [3]:
import pandas as pd
import numpy as np
from deltalake import DeltaTable, write_deltalake
from datetime import datetime, timezone
import os

# ===============================================================
# 🔧 Configuración MinIO y rutas usando variables de entorno
# ===============================================================

storage_options = {
    "AWS_ENDPOINT_URL": os.environ["AWS_ENDPOINT_URL"],
    "AWS_ACCESS_KEY_ID": os.environ["AWS_ACCESS_KEY_ID"],
    "AWS_SECRET_ACCESS_KEY": os.environ["AWS_SECRET_ACCESS_KEY"],
    "AWS_ALLOW_HTTP": "true"
}

# Nombre del bucket
bkt = "sebastianmedina-bucket"

# Rutas para TP2 (Silver y Bronze)
bronze_weather_path   = os.environ["BRONZE_INC"]   # Datos horarios meteorológicos
bronze_meta_path      = os.environ["BRONZE_FULL"]  # Datos completos / metadata
silver_weather_path   = os.environ["SILVER_WEATHER"]      # Silver horario
silver_daily_avg_path = os.environ["SILVER_DAILY_AVG"]    # Silver agregado diario

print("✅ Configuración MinIO y rutas cargadas")


✅ Configuración MinIO y rutas cargadas


### 🛠️ Funciones para manejo de Delta Lake

- `read_delta_to_pandas(delta_path)`  
  Lee una tabla Delta desde MinIO y devuelve un DataFrame. Maneja errores y retorna un DataFrame vacío si falla.

- `save_pandas_to_delta(df, delta_path, mode="overwrite")`  
  Guarda un DataFrame en Delta Lake sobre MinIO, permitiendo modo `overwrite` o `append`.

- `show_basic_info(df, name="DataFrame")`  
  Muestra información rápida del DataFrame: cantidad de filas y columnas, tipos de columnas y nulos por columna.


In [4]:
def read_delta_to_pandas(delta_path: str):
    """Lee una tabla Delta desde MinIO y devuelve un DataFrame."""
    try:
        dt = DeltaTable(delta_path, storage_options=storage_options)
        df = dt.to_pandas()
        print(f"✅ Leído desde {delta_path} -> {df.shape[0]} filas, {df.shape[1]} columnas")
        return df
    except Exception as e:
        print(f"❌ Error leyendo {delta_path}: {e}")
        return pd.DataFrame()

def save_pandas_to_delta(df: pd.DataFrame, delta_path: str, mode="overwrite"):
    """Guarda un DataFrame en Delta Lake sobre MinIO."""
    try:
        write_deltalake(delta_path, df, mode=mode, storage_options=storage_options, schema_mode="merge")
        print(f"✅ Guardado en Delta: {delta_path} | mode={mode} | filas={len(df)}")
    except Exception as e:
        print(f"❌ Error guardando en {delta_path}: {e}")

def show_basic_info(df: pd.DataFrame, name="DataFrame"):
    """Muestra shape, tipos de columnas y nulos de un DataFrame."""
    print(f"--- {name} ---")
    print("Shape:", df.shape)
    print("Tipos:\n", df.dtypes)
    print("Nulos por columna:\n", df.isna().sum())
    print()


# 📥 Lectura desde Bronze

En esta celda cargamos los datasets horarios almacenados en la capa **Bronze**:

- **`weather_hourly`** → datos meteorológicos.
- **`air_quality_hourly`** → datos de calidad del aire.

Se utiliza la función `read_delta_to_pandas` para leer los archivos desde MinIO.  
En caso de cualquier error (archivo inexistente, formato incorrecto o problemas de conexión), se devuelve un **DataFrame vacío** para que el pipeline continúe sin interrumpirse.


In [5]:
# ===============================================================
# 📥 Lectura desde Bronze (WEATHER + AIR QUALITY)
# ===============================================================

# --- WEATHER HOURLY ---
# Leemos el dataset horario de meteorología desde la carpeta específica 'weather'
# dentro de Bronze incremental. Cada carpeta contiene su propio _delta_log y particiones por fecha.
df_weather = read_delta_to_pandas(bronze_weather_path + "weather/")
display(df_weather.head())  # Mostramos primeras filas para inspección rápida

# --- AIR QUALITY HOURLY ---
# Leemos el dataset horario de calidad del aire desde la carpeta específica 'air_quality'
# dentro de Bronze incremental. Esto garantiza que DeltaTable encuentre los archivos y logs correctos.
df_air_quality = read_delta_to_pandas(bronze_weather_path + "air_quality/")
display(df_air_quality.head())  # Mostramos primeras filas para inspección rápida


✅ Leído desde s3://sebastianmedina-bucket/bronze/incremental/weather/ -> 24 filas, 12 columnas


,time,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,weathercode,pressure_msl,cloudcover,visibility,windspeed_10m,winddirection_10m,date
0,2025-12-03 00:00:00,19.0,90,21.0,0.0,1,1012.2,7,24140.0,4.6,72,2025-12-03
1,2025-12-03 01:00:00,18.7,90,20.7,0.0,0,1012.3,2,24140.0,3.2,63,2025-12-03
2,2025-12-03 02:00:00,18.5,90,20.3,0.0,0,1012.0,0,24140.0,3.6,37,2025-12-03
3,2025-12-03 03:00:00,18.3,91,20.1,0.0,1,1011.9,7,24140.0,3.8,17,2025-12-03
4,2025-12-03 04:00:00,18.3,92,20.1,0.0,1,1011.8,35,24140.0,4.0,10,2025-12-03


✅ Leído desde s3://sebastianmedina-bucket/bronze/incremental/air_quality/ -> 24 filas, 8 columnas


,time,pm10,pm2_5,carbon_monoxide,ozone,nitrogen_dioxide,sulphur_dioxide,date
0,2025-12-03 00:00:00,9.0,8.6,143.0,42.0,25.9,4.8,2025-12-03
1,2025-12-03 01:00:00,8.6,8.2,132.0,43.0,23.6,4.7,2025-12-03
2,2025-12-03 02:00:00,8.4,8.0,118.0,42.0,21.3,4.6,2025-12-03
3,2025-12-03 03:00:00,8.2,7.8,111.0,41.0,19.9,4.6,2025-12-03
4,2025-12-03 04:00:00,8.4,8.1,115.0,37.0,20.1,4.9,2025-12-03


#🩺 Análisis inicial
Inspeccionamos el DataFrame:
- cantidad de filas y columnas
- columnas con nulos
- tipos de datos sospechosos


In [6]:
# Mostramos información básica del DataFrame de WEATHER
show_basic_info(df_weather, "Bronze - weather_hourly")

# Listamos los nombres de las columnas para inspección rápida
print("Columnas:", df_weather.columns.tolist())

# Mostramos información básica del DataFrame de AIR QUALITY
show_basic_info(df_air_quality, "Bronze - air_quality_hourly")
print("Columnas:", df_air_quality.columns.tolist())


--- Bronze - weather_hourly ---
Shape: (24, 12)
Tipos:
 time                    datetime64[us]
temperature_2m                 float64
relative_humidity_2m             int64
apparent_temperature           float64
precipitation                  float64
weathercode                      int64
pressure_msl                   float64
cloudcover                       int64
visibility                     float64
windspeed_10m                  float64
winddirection_10m                int64
date                            object
dtype: object
Nulos por columna:
 time                    0
temperature_2m          0
relative_humidity_2m    0
apparent_temperature    0
precipitation           0
weathercode             0
pressure_msl            0
cloudcover              0
visibility              0
windspeed_10m           0
winddirection_10m       0
date                    0
dtype: int64

Columnas: ['time', 'temperature_2m', 'relative_humidity_2m', 'apparent_temperature', 'precipitation', 'weathercode',

#🧹 Limpieza de valores nulos

Aplicamos una estrategia simple y segura para manejar valores faltantes:

- **Columnas numéricas:** se rellenan con la **mediana**, que es más robusta que el promedio ante valores atípicos.
- **Columnas categóricas:** se completan con el valor **'unknown'** para mantener consistencia y evitar perder filas.
- **No eliminamos filas completas**, ya que la cantidad de nulos en este dataset es baja y preferimos conservar la mayor parte de la información posible.

Esto garantiza un dataset limpio y estable para los pasos posteriores del pipeline.



In [7]:
def clean_fill_missing(df: pd.DataFrame):
    """
    Limpia nulos numéricos y categóricos, y unifica nombres de columnas clave.

    - Columnas numéricas → rellenar con la mediana.
    - Columnas categóricas → rellenar con 'unknown'.
    - Renombrar columnas relevantes de WEATHER para unificar nombres.
    """
    if df.empty:
        print("⚠️ DataFrame vacío, no se aplica limpieza")
        return df

    nulos_antes = df.isna().sum()

    # Renombrar columnas clave
    rename_map = {}
    if "temperature_2m" in df.columns: rename_map["temperature_2m"] = "temperature"
    if "relative_humidity_2m" in df.columns: rename_map["relative_humidity_2m"] = "humidity"
    if "windspeed_10m" in df.columns: rename_map["windspeed_10m"] = "wind_speed"
    df = df.rename(columns=rename_map)

    # Columnas numéricas
    numeric_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    for col in numeric_cols:
        med = df[col].median(skipna=True)
        df[col] = df[col].fillna(med if not pd.isna(med) else 0)

    # Columnas categóricas
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].fillna("unknown")

    nulos_despues = df.isna().sum()
    print("Nulos antes:\n", nulos_antes[nulos_antes>0])
    print("\nNulos después:\n", nulos_despues[nulos_despues>0])

    return df

# Aplicar limpieza a los DataFrames cargados desde Bronze
df_weather = clean_fill_missing(df_weather)
df_air_quality = clean_fill_missing(df_air_quality)

Nulos antes:
 Series([], dtype: int64)

Nulos después:
 Series([], dtype: int64)
Nulos antes:
 Series([], dtype: int64)

Nulos después:
 Series([], dtype: int64)


### 🧽 Eliminación de duplicados

En esta celda removemos registros duplicados para asegurar consistencia en el dataset:

- Si existen las columnas **`time`** y **`station_id`**, se consideran como clave compuesta —dos registros con la misma combinación se tratan como duplicados.
- Si esas columnas no están presentes (por ejemplo, en los datos de calidad del aire), se evalúan **todas las columnas** para identificar filas idénticas.

Aunque en muchos casos la API no devuelve duplicados, esta validación garantiza que el dataset Silver quede limpio y preparado para análisis posteriores.


In [8]:
def remove_duplicates(df: pd.DataFrame):
    """
    Elimina duplicados basados en columnas lógicas o todas las columnas.

    - Si existen columnas 'time' y 'station_id', se usan como clave compuesta.
    - Si no existen, se consideran todas las columnas para detectar duplicados.
    """
    if df.empty:
        print("⚠️ DataFrame vacío, no se eliminan duplicados")
        return df

    dup_subset = ["time", "station_id"] if "time" in df.columns and "station_id" in df.columns else df.columns.tolist()

    before = df.shape[0]
    df = df.drop_duplicates(subset=dup_subset)
    after = df.shape[0]

    print(f"Duplicados eliminados: {before - after}")
    return df

# Aplicar limpieza de duplicados a los DataFrames cargados desde Bronze
df_weather = remove_duplicates(df_weather)
df_air_quality = remove_duplicates(df_air_quality)


Duplicados eliminados: 0
Duplicados eliminados: 0


# 🕒 Conversión de fechas y categorías

En esta celda transformamos los campos temporales y categóricos del dataset adaptado a los nuevos endpoints:

- **Convertimos columnas de fecha y hora a `datetime`** para permitir filtrado, ordenamiento y resampling.
- **Creamos columnas auxiliares**:
  - `extract_date` → solo la fecha del registro.
  - `extract_hour` → hora del registro.
- **Convertimos columnas categóricas a tipo `category`**, optimizando memoria y velocidad de agrupamiento.

Estas transformaciones aseguran que el dataset Silver esté listo para limpieza, agregación y análisis posteriores, tanto para WEATHER como para AIR QUALITY.


In [9]:
def convert_and_format_dates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte fechas a datetime, crea columnas auxiliares y optimiza columnas categóricas.

    Parámetros:
    - df: DataFrame que contiene datos horarios de WEATHER o AIR QUALITY.

    Funcionalidad:
    - Detecta columna temporal ('time' o 'datetime') y la convierte a datetime.
    - Crea columnas auxiliares:
        * extract_date → fecha en formato 'YYYY-MM-DD'
        * extract_hour → hora como string
    - Convierte columnas de identificación/categóricas a tipo 'category':
        'id', 'station_id', 'location', 'name'
    - Si no encuentra columna temporal, asigna fecha y hora actual.
    """
    from datetime import datetime, timezone

    time_col = "time" if "time" in df.columns else ("datetime" if "datetime" in df.columns else None)
    now = datetime.now(timezone.utc)

    if time_col:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
        df["extract_date"] = df[time_col].dt.date.astype(str)
        df["extract_hour"] = df[time_col].dt.hour.astype(str)
    else:
        df["extract_date"] = now.strftime("%Y-%m-%d")
        df["extract_hour"] = now.strftime("%H")

    # Optimiza columnas categóricas
    for col in ["id", "station_id", "location", "name"]:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df
# Procesar fechas y categorías
df_weather = convert_and_format_dates(df_weather)
df_air_quality = convert_and_format_dates(df_air_quality)

# Mostrar primeras filas para verificar
display(df_weather.head())
display(df_air_quality.head())



,time,temperature,humidity,apparent_temperature,precipitation,weathercode,pressure_msl,cloudcover,visibility,wind_speed,winddirection_10m,date,extract_date,extract_hour
0,2025-12-03 00:00:00,19.0,90,21.0,0.0,1,1012.2,7,24140.0,4.6,72,2025-12-03,2025-12-03,0
1,2025-12-03 01:00:00,18.7,90,20.7,0.0,0,1012.3,2,24140.0,3.2,63,2025-12-03,2025-12-03,1
2,2025-12-03 02:00:00,18.5,90,20.3,0.0,0,1012.0,0,24140.0,3.6,37,2025-12-03,2025-12-03,2
3,2025-12-03 03:00:00,18.3,91,20.1,0.0,1,1011.9,7,24140.0,3.8,17,2025-12-03,2025-12-03,3
4,2025-12-03 04:00:00,18.3,92,20.1,0.0,1,1011.8,35,24140.0,4.0,10,2025-12-03,2025-12-03,4


,time,pm10,pm2_5,carbon_monoxide,ozone,nitrogen_dioxide,sulphur_dioxide,date,extract_date,extract_hour
0,2025-12-03 00:00:00,9.0,8.6,143.0,42.0,25.9,4.8,2025-12-03,2025-12-03,0
1,2025-12-03 01:00:00,8.6,8.2,132.0,43.0,23.6,4.7,2025-12-03,2025-12-03,1
2,2025-12-03 02:00:00,8.4,8.0,118.0,42.0,21.3,4.6,2025-12-03,2025-12-03,2
3,2025-12-03 03:00:00,8.2,7.8,111.0,41.0,19.9,4.6,2025-12-03,2025-12-03,3
4,2025-12-03 04:00:00,8.4,8.1,115.0,37.0,20.1,4.9,2025-12-03,2025-12-03,4


# 🏷️ Renombrado de columnas

En esta celda unificamos y estandarizamos los nombres de columnas para que el dataset **Silver** tenga nombres consistentes y fáciles de usar:

- **temperature → temp_c**: se aclara que el valor está en grados Celsius.  
- **humidity → rel_humidity_pct**: se especifica que representa humedad relativa en porcentaje.

Este paso asegura claridad en el esquema y evita ambigüedades para análisis posteriores, aplicable tanto a WEATHER como a AIR QUALITY.


In [10]:
def rename_columns(df: pd.DataFrame):
    """Renombra columnas para estandarizar nombres en Silver, compatible con WEATHER y AIR QUALITY."""
    rename_map = {}
    if "temperature" in df.columns and "temp_c" not in df.columns:
        rename_map["temperature"] = "temp_c"
    if "humidity" in df.columns and "rel_humidity_pct" not in df.columns:
        rename_map["humidity"] = "rel_humidity_pct"

    if rename_map:
        df = df.rename(columns=rename_map)

    print("Renombradas columnas:", rename_map)
    return df

# Ejemplo de uso: aplicar al dataset cargado previamente
df_weather = rename_columns(df_weather)
df_air_quality = rename_columns(df_air_quality)

# Mostrar primeras filas para verificación
display(df_weather.head())



Renombradas columnas: {'temperature': 'temp_c', 'humidity': 'rel_humidity_pct'}
Renombradas columnas: {}


,time,temp_c,rel_humidity_pct,apparent_temperature,precipitation,weathercode,pressure_msl,cloudcover,visibility,wind_speed,winddirection_10m,date,extract_date,extract_hour
0,2025-12-03 00:00:00,19.0,90,21.0,0.0,1,1012.2,7,24140.0,4.6,72,2025-12-03,2025-12-03,0
1,2025-12-03 01:00:00,18.7,90,20.7,0.0,0,1012.3,2,24140.0,3.2,63,2025-12-03,2025-12-03,1
2,2025-12-03 02:00:00,18.5,90,20.3,0.0,0,1012.0,0,24140.0,3.6,37,2025-12-03,2025-12-03,2
3,2025-12-03 03:00:00,18.3,91,20.1,0.0,1,1011.9,7,24140.0,3.8,17,2025-12-03,2025-12-03,3
4,2025-12-03 04:00:00,18.3,92,20.1,0.0,1,1011.8,35,24140.0,4.0,10,2025-12-03,2025-12-03,4


# ➕ Columnas derivadas (mejoradas)

En esta celda agregamos nuevas columnas útiles para análisis de WEATHER y AIR QUALITY:

- **temp_above_30** → indica si la temperatura supera 30°C.  
- **temp_f** → temperatura en Fahrenheit.  
- **dew_point_c_est** → punto de rocío estimado según temperatura y humedad.  
- **rolling_mean_temp_3h** → promedio móvil de temperatura de 3 horas.  
- **heat_index_c** → sensación térmica estimada.  
- **wind_kmh** → velocidad del viento convertida a km/h.  
- **temp_category** → clasificación simple de temperatura (frío / templado / caliente / muy caliente).  
- **period_day** → periodo del día (madrugada, mañana, tarde, noche).  
- **weekday** → día de la semana.

Estas columnas enriquecen el dataset y facilitan análisis, visualizaciones y agregaciones posteriores.


In [11]:
def create_derived_columns_enhanced(df: pd.DataFrame):
    """Crea columnas derivadas mejoradas para análisis meteorológico."""

    # Temperatura y humedad
    if "temp_c" not in df.columns and "temperature" in df.columns:
        df["temp_c"] = df["temperature"]
    if "temp_c" in df.columns:
        df["temp_above_30"] = df["temp_c"] > 30
        df["temp_f"] = df["temp_c"] * 9/5 + 32
    else:
        df["temp_above_30"] = False
        df["temp_f"] = np.nan

    if "temp_c" in df.columns and "rel_humidity_pct" in df.columns:
        df["dew_point_c_est"] = df["temp_c"] - ((100 - df["rel_humidity_pct"]) / 5)
        df["heat_index_c"] = df["temp_c"] + 0.33*df["rel_humidity_pct"] - 0.7
    else:
        df["dew_point_c_est"] = np.nan
        df["heat_index_c"] = np.nan

    # Velocidad del viento en km/h
    if "wind_speed" in df.columns:
        df["wind_kmh"] = df["wind_speed"] * 3.6
    else:
        df["wind_kmh"] = np.nan

    # Rolling mean de 3 horas
    if "temp_c" in df.columns and "time" in df.columns:
        df = df.sort_values("time")
        df["rolling_mean_temp_3h"] = df["temp_c"].rolling(window=3, min_periods=1).mean()
    else:
        df["rolling_mean_temp_3h"] = np.nan

    # Clasificación de temperatura
    bins = [0, 15, 25, 30, 50]
    labels = ["Frío", "Templado", "Caliente", "Muy caliente"]
    if "temp_c" in df.columns:
        df["temp_category"] = pd.cut(df["temp_c"], bins=bins, labels=labels)
    else:
        df["temp_category"] = "unknown"

    # Periodo del día
    def period_of_day(hour):
        if 0 <= hour < 6: return "Madrugada"
        elif 6 <= hour < 12: return "Mañana"
        elif 12 <= hour < 18: return "Tarde"
        else: return "Noche"

    if "extract_hour" in df.columns:
        df["period_day"] = df["extract_hour"].astype(int).apply(period_of_day)
    else:
        df["period_day"] = "unknown"

    # Día de la semana
    if "extract_date" in df.columns:
        df["weekday"] = pd.to_datetime(df["extract_date"]).dt.day_name()
    else:
        df["weekday"] = "unknown"

    print("✅ Columnas derivadas mejoradas creadas")
    return df

# Aplicar a los DataFrames cargados
df_weather = create_derived_columns_enhanced(df_weather)
df_air_quality = create_derived_columns_enhanced(df_air_quality)

# Mostrar primeras filas para verificación
display(df_weather.head())



✅ Columnas derivadas mejoradas creadas
✅ Columnas derivadas mejoradas creadas


,time,temp_c,rel_humidity_pct,apparent_temperature,precipitation,weathercode,pressure_msl,cloudcover,visibility,wind_speed,...,extract_hour,temp_above_30,temp_f,dew_point_c_est,heat_index_c,wind_kmh,rolling_mean_temp_3h,temp_category,period_day,weekday
0,2025-12-03 00:00:00,19.0,90,21.0,0.0,1,1012.2,7,24140.0,4.6,...,0,False,66.20,17.0,48.00,16.56,19.000000,Templado,Madrugada,Wednesday
1,2025-12-03 01:00:00,18.7,90,20.7,0.0,0,1012.3,2,24140.0,3.2,...,1,False,65.66,16.7,47.70,11.52,18.850000,Templado,Madrugada,Wednesday
2,2025-12-03 02:00:00,18.5,90,20.3,0.0,0,1012.0,0,24140.0,3.6,...,2,False,65.30,16.5,47.50,12.96,18.733333,Templado,Madrugada,Wednesday
3,2025-12-03 03:00:00,18.3,91,20.1,0.0,1,1011.9,7,24140.0,3.8,...,3,False,64.94,16.5,47.63,13.68,18.500000,Templado,Madrugada,Wednesday
4,2025-12-03 04:00:00,18.3,92,20.1,0.0,1,1011.8,35,24140.0,4.0,...,4,False,64.94,16.7,47.96,14.40,18.366667,Templado,Madrugada,Wednesday


#📈 Agregación diaria mejorada

Generamos un nuevo dataset diario a partir de los datos horarios. Calculamos:

- **avg_temp_c** → temperatura promedio del día  
- **median_temp_c** → temperatura mediana del día  
- **std_temp_c** → variación (desvío estándar)  
- **max_temp_c / min_temp_c** → máximas y mínimas diarias  
- **avg_wind_speed** → viento promedio  
- **hours_hot** → cantidad de horas con temperatura > 30°C  
- **hours_rain** → cantidad de horas con precipitación > 0 mm  

Estas métricas permiten analizar tendencias climáticas diarias de forma compacta.


In [12]:
def daily_aggregates_enhanced(df: pd.DataFrame):
    """
    Genera un DataFrame diario agregado a partir de los datos horarios.

    Métricas calculadas:
    - Promedio, mediana, std, máximo y mínimo de temperatura.
    - Velocidad promedio del viento.
    - Cantidad de horas con temperatura > 30°C.
    - Cantidad de horas con precipitación.
    """
    if "temp_c" in df.columns and "extract_date" in df.columns:
        daily = df.groupby("extract_date").agg(
            avg_temp_c=("temp_c", "mean"),
            median_temp_c=("temp_c", "median"),
            std_temp_c=("temp_c", "std"),
            max_temp_c=("temp_c", "max"),
            min_temp_c=("temp_c", "min"),
            avg_wind_speed=("wind_speed", "mean") if "wind_speed" in df.columns else pd.NamedAgg(column="temp_c", aggfunc="mean"),
            hours_hot=("temp_above_30", "sum") if "temp_above_30" in df.columns else pd.NamedAgg(column="temp_c", aggfunc=lambda x: 0),
            hours_rain=("precipitation", lambda x: (x>0).sum()) if "precipitation" in df.columns else pd.NamedAgg(column="temp_c", aggfunc=lambda x: 0)
        ).reset_index()

        # Redondear métricas de temperatura a 2 decimales
        daily[["avg_temp_c", "median_temp_c"]] = daily[["avg_temp_c", "median_temp_c"]].round(2)
        print("✅ Agregados diarios generados")
        return daily
    else:
        print("❌ Columnas necesarias para agregación diaria no encontradas")
        return pd.DataFrame()

# Aplicar función al DataFrame correcto
daily_weather = daily_aggregates_enhanced(df_weather)
display(daily_weather.head())

# Si quieres, también puedes aplicar a air quality, aunque normalmente no tiene temp_c
# daily_air_quality = daily_aggregates_enhanced(df_air_quality)
# display(daily_air_quality.head())


✅ Agregados diarios generados


,extract_date,avg_temp_c,median_temp_c,std_temp_c,max_temp_c,min_temp_c,avg_wind_speed,hours_hot,hours_rain
0,2025-12-03,24.37,24.35,4.904965,31.0,18.3,6.5875,4,0


# 🪙 Guardado en Silver

En esta celda escribimos los resultados procesados en la capa **Silver**:

- **weather_hourly** → dataset limpio y transformado, con columnas derivadas.  
- **daily averages** → dataset agregado con métricas diarias.

Ambos se guardan en formato **Delta Lake**, usando `mode="overwrite"` para reemplazar la versión anterior y mantener siempre la más reciente.


In [13]:
# Guardado de datasets procesados en Silver

# Silver horario
save_pandas_to_delta(df_weather, silver_weather_path, mode="overwrite")

# Silver agregado diario
if not daily_weather.empty:
    save_pandas_to_delta(daily_weather, silver_daily_avg_path, mode="overwrite")


✅ Guardado en Delta: s3://sebastianmedina-bucket/silver/weather_cleaned/ | mode=overwrite | filas=24
✅ Guardado en Delta: s3://sebastianmedina-bucket/silver/weather_daily_avg/ | mode=overwrite | filas=1
